# SQL Agent for Spider text-to-SQL benchmark

This notebook demonstrates a basic SQL agent that translates natural language questions into SQL queries.

## Environment

For this demo, we use a SQLite database environment based on a standard text-to-sql benchmark called [Spider](https://yale-lily.github.io/spider). The environment provides a gym-like interface and can be used as follows.

In [1]:
# %pip install spider-env
import sys
sys.path.append('/home/simges/autogen/autogen/')

import json
import os
from typing import Annotated, Dict

from spider_env import SpiderEnv

from autogen import ConversableAgent, UserProxyAgent, config_list_from_json

gym = SpiderEnv()

# Randomly select a question from Spider
observation, info = gym.reset()

Loading cached Spider dataset from /home/simges/.cache/spider
Schema file not found for /home/simges/.cache/spider/spider/database/twitter_1
Schema file not found for /home/simges/.cache/spider/spider/database/flight_4
Schema file not found for /home/simges/.cache/spider/spider/database/small_bank_1
Schema file not found for /home/simges/.cache/spider/spider/database/icfp_1
Schema file not found for /home/simges/.cache/spider/spider/database/epinions_1
Schema file not found for /home/simges/.cache/spider/spider/database/company_1
Schema file not found for /home/simges/.cache/spider/spider/database/chinook_1
/home/simges/.cache/spider/spider/database
music_4/music_4.sqlite


In [2]:
# The natural language question
question = observation["instruction"]
print(question)

Find the famous titles of artists that do not have any volume.


In [3]:
# The schema of the corresponding database
schema = info["schema"]
print(schema)

CREATE TABLE "artist" (
"Artist_ID" int,
"Artist" text,
"Age" int,
"Famous_Title" text,
"Famous_Release_date" text,
PRIMARY KEY ("Artist_ID")
);
CREATE TABLE "volume" (
"Volume_ID" int,
"Volume_Issue" text,
"Issue_Date" text,
"Weeks_on_Top" real,
"Song" text,
"Artist_ID" int,
PRIMARY KEY ("Volume_ID"),
FOREIGN KEY ("Artist_ID") REFERENCES "artist"("Artist_ID")
);
CREATE TABLE "music_festival" (
"ID" int,
"Music_Festival" text,
"Date_of_ceremony" text,
"Category" text,
"Volume" int,
"Result" text,
PRIMARY KEY ("ID"),
FOREIGN KEY ("Volume") REFERENCES "volume"("Volume_ID")
);



## Agent Implementation

Using AutoGen, a SQL agent can be implemented with a ConversableAgent. The gym environment executes the generated SQL query and the agent can take execution results as feedback to improve its generation in multiple rounds of conversations.

In [10]:
os.environ["AUTOGEN_USE_DOCKER"] = "False"
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

def check_termination(msg: Dict):
    if "tool_responses" not in msg:
        return False
    json_str = msg["tool_responses"][0]["content"]
    obj = json.loads(json_str)
    return "error" not in obj or obj["error"] is None and obj["reward"] == 1


sql_writer = ConversableAgent(
    name="sql_writer",
    llm_config=config_list[1],
    system_message="You are good at writing SQL queries.",
    is_termination_msg=check_termination,
)
sql_executor = ConversableAgent(
    name="sql_executor",
    llm_config=config_list[1],
    system_message="You are supposed to execute sql queries generated by sql_writer",
    is_termination_msg=check_termination,
)
user_proxy = UserProxyAgent(name="user_proxy",
                            human_input_mode="NEVER", 
                            code_execution_config=False,
                            llm_config=config_list[1],
                            max_consecutive_auto_reply=5)


@user_proxy.register_for_execution()
@sql_executor.register_for_llm(description="Function for executing SQL query and returning a response")
def execute_sql(
    reflection: Annotated[str, "Think about what to do"], sql: Annotated[str, "SQL query"]
) -> Annotated[Dict[str, str], "Dictionary with keys 'result' and 'error'"]:
    print("xxxxxxxxxxxxxxxxxxxxxxx " + str(sql))
    observation, reward, _, _, info = gym.step(sql)
    error = observation["feedback"]["error"]
    if not error and reward == 0:
        error = "The SQL query returned an incorrect result"
    if error:
        return {
            "error": error,
            "wrong_result": observation["feedback"]["result"],
            "correct_result": info["gold_result"],
        }
    else:
        return {
            "result": observation["feedback"]["result"],
        }
print("funcccccccccccs: " + str(sql_executor.llm_config["tools"]))

import autogen
groupchat = autogen.GroupChat(
    agents=[user_proxy, sql_writer, sql_executor],
    messages=[],
    max_round=12,
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=config_list[1])

[runtime logging] log_new_wrapper: autogen logger is None
[runtime logging] log_new_client: autogen logger is None
[runtime logging] log_new_agent: autogen logger is None
[runtime logging] log_new_wrapper: autogen logger is None
[runtime logging] log_new_client: autogen logger is None
[runtime logging] log_new_agent: autogen logger is None
[runtime logging] log_new_wrapper: autogen logger is None
[runtime logging] log_new_client: autogen logger is None
[runtime logging] log_new_agent: autogen logger is None
[runtime logging] log_new_agent: autogen logger is None
[runtime logging] log_new_wrapper: autogen logger is None
[runtime logging] log_new_client: autogen logger is None
[runtime logging] log_new_wrapper: autogen logger is None
[runtime logging] log_new_client: autogen logger is None
[runtime logging] log_new_agent: autogen logger is None
[runtime logging] log_new_agent: autogen logger is None


func nameeeeeee: execute_sql
FUNCTION MAPPPPPPP
execute_sql: <function execute_sql at 0x7f7cb81b2e80>
funcccccccccccs: [{'type': 'function', 'function': {'description': 'Function for executing SQL query and returning a response', 'name': 'execute_sql', 'parameters': {'type': 'object', 'properties': {'reflection': {'type': 'string', 'description': 'Think about what to do'}, 'sql': {'type': 'string', 'description': 'SQL query'}}, 'required': ['reflection', 'sql']}}}]


The agent can then take as input the schema and the text question, and generate the SQL query.

In [11]:
message = f"""Below is the schema for a SQL database:
{schema}
Generate a SQL query to answer the following question:
{question}
"""

# Start chatting with the boss as this is the user proxy agent.
user_proxy.initiate_chat(manager, message=message)

vvvvvvvvvvvvvvvvvv
Below is the schema for a SQL database:
CREATE TABLE "artist" (
"Artist_ID" int,
"Artist" text,
"Age" int,
"Famous_Title" text,
"Famous_Release_date" text,
PRIMARY KEY ("Artist_ID")
);
CREATE TABLE "volume" (
"Volume_ID" int,
"Volume_Issue" text,
"Issue_Date" text,
"Weeks_on_Top" real,
"Song" text,
"Artist_ID" int,
PRIMARY KEY ("Volume_ID"),
FOREIGN KEY ("Artist_ID") REFERENCES "artist"("Artist_ID")
);
CREATE TABLE "music_festival" (
"ID" int,
"Music_Festival" text,
"Date_of_ceremony" text,
"Category" text,
"Volume" int,
"Result" text,
PRIMARY KEY ("ID"),
FOREIGN KEY ("Volume") REFERENCES "volume"("Volume_ID")
);

Generate a SQL query to answer the following question:
Find the famous titles of artists that do not have any volume.

user_proxy (to chat_manager):

Below is the schema for a SQL database:
CREATE TABLE "artist" (
"Artist_ID" int,
"Artist" text,
"Age" int,
"Famous_Title" text,
"Famous_Release_date" text,
PRIMARY KEY ("Artist_ID")
);
CREATE TABLE "volume" (


[runtime logging] log_chat_completion: autogen logger is None


sql_writer (to chat_manager):

 To find the famous titles of artists who do not have any associated volumes, we can use a LEFT JOIN and check for null values in the "Volume_ID" column of the "volume" table. Here is the SQL query to achieve that:

```sql
SELECT Artist."Famous_Title"
FROM artist AS Artist
LEFT JOIN volume AS Volume ON Artist."Artist_ID" = Volume."Artist_ID"
WHERE Volume."Volume_ID" IS NULL;
```

This query will return the famous titles of artists who do not have any associated volumes. If you need the full artist name and other details, replace "Artist.\*' with the specific columns as needed in the SELECT clause.

--------------------------------------------------------------------------------
reply funcccc : <function ConversableAgent.a_check_termination_and_human_reply at 0x7f7cbcb64c20>
reply funcccc : <function ConversableAgent.check_termination_and_human_reply at 0x7f7cbcb64b80>

>>>>>>>> USING AUTO REPLY...
reply funcccc : <function ConversableAgent.a_generate_func

[runtime logging] log_chat_completion: autogen logger is None


sql_executor (to chat_manager):

[TOOL_CALLS] [{"name":"execute_sql","arguments":{"reflection":"Find the famous titles of artists that do not have any volume","sql":"SELECT Artist.\* FROM artist AS Artist LEFT JOIN volume AS Volume ON Artist.\* = Volume.\* WHERE Volume.\* IS NULL;"}}]

--------------------------------------------------------------------------------
reply funcccc : <function ConversableAgent.a_check_termination_and_human_reply at 0x7f7cbcb64c20>
reply funcccc : <function ConversableAgent.check_termination_and_human_reply at 0x7f7cbcb64b80>
reply funcccc : <function ConversableAgent.a_generate_function_call_reply at 0x7f7cbcb64860>
reply funcccc : <function ConversableAgent.generate_function_call_reply at 0x7f7cbcb647c0>
messageee : {'content': '[TOOL_CALLS] [{"name":"execute_sql","arguments":{"reflection":"Find the famous titles of artists that do not have any volume","sql":"SELECT Artist.\\* FROM artist AS Artist LEFT JOIN volume AS Volume ON Artist.\\* = Volume.\\* WH

...

[runtime logging] log_chat_completion: autogen logger is None


sql_writer (to chat_manager):

 [{"name":"execute_sql","arguments":{"reflection":"Find the famous titles of artists that do not have any volume, including their full name, age and filter only for those artists without associated volumes in the 'volume' table","sql":"SELECT Artist.\* FROM artist AS Artist LEFT JOIN volume AS Volume ON Artist.\* = Volume.\* WHERE Volume.\* IS NULL AND Artist.\* NOT IN (SELECT Artist_ID FROM volume);"}}]

 Here's the revised SQL query as you requested, filtering out artists who do have associated volumes:

```sql
SELECT Artist."Artist", Artist."Age", Artist."Famous_Title"
FROM artist AS Artist
LEFT JOIN volume AS Volume ON Artist."Artist_ID" = Volume."Artist_ID"
WHERE Volume."Volume_ID" IS NULL AND Artist."Artist_ID" NOT IN (SELECT Artist_ID FROM volume);
```

This query will return the full name, age, and famous titles of artists who do not have any associated volumes.

--------------------------------------------------------------------------------
repl

[runtime logging] log_chat_completion: autogen logger is None


sql_executor (to chat_manager):

 [{"name":"execute_sql","arguments":{"reflection":"Find the famous titles of artists that do not have any volume, including their full name, age and filter only for those artists without associated volumes in the 'volume' table","sql":"SELECT Artist.\* FROM artist AS Artist LEFT JOIN volume AS Volume ON Artist.\* = Volume.\* WHERE Volume.\* IS NULL AND Artist.\* NOT IN (SELECT Artist_ID FROM volume);"}}]

--------------------------------------------------------------------------------
replyyyy None
sender.name user_proxy


ChatResult(chat_id=None, chat_history=[{'content': 'Below is the schema for a SQL database:\nCREATE TABLE "artist" (\n"Artist_ID" int,\n"Artist" text,\n"Age" int,\n"Famous_Title" text,\n"Famous_Release_date" text,\nPRIMARY KEY ("Artist_ID")\n);\nCREATE TABLE "volume" (\n"Volume_ID" int,\n"Volume_Issue" text,\n"Issue_Date" text,\n"Weeks_on_Top" real,\n"Song" text,\n"Artist_ID" int,\nPRIMARY KEY ("Volume_ID"),\nFOREIGN KEY ("Artist_ID") REFERENCES "artist"("Artist_ID")\n);\nCREATE TABLE "music_festival" (\n"ID" int,\n"Music_Festival" text,\n"Date_of_ceremony" text,\n"Category" text,\n"Volume" int,\n"Result" text,\nPRIMARY KEY ("ID"),\nFOREIGN KEY ("Volume") REFERENCES "volume"("Volume_ID")\n);\n\nGenerate a SQL query to answer the following question:\nFind the famous titles of artists that do not have any volume.\n', 'role': 'assistant'}, {'content': ' To find the famous titles of artists who do not have any associated volumes, we can use a LEFT JOIN and check for null values in the "Vol